---
title: Running personalized Enformer pipeline on Br (round 2)
date: 11/02/2023
author: Sabrina Mi
---

## Selecting Genes

We selected 868 genes in the first round, we are aiming to double the gene set.

In [1]:
import pandas as pd
import numpy as np

In [2]:
tpm = pd.read_csv("/home/s1mi/enformer_rat_data/expression_data/Brain.rn7.expr.tpm.bed", header = 0, sep="\t",  index_col= 'gene_id')
tpm_var = tpm.iloc[:, 3:].var(axis=1)
tpm_threshold = tpm_var.quantile(0.75)
## subset of genes with high observed expression variation
high_tpm_var_genes = set(tpm[tpm_var> tpm_threshold].index)

/tmp/ipykernel_79532/161308317.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  tpm = pd.read_csv("/home/s1mi/enformer_rat_data/expression_data/Brain.rn7.expr.tpm.bed", header = 0, sep="\t",  index_col= 'gene_id')


In [3]:
eqtl = pd.read_csv("/home/s1mi/enformer_rat_data/annotation/Brain.rn7.cis_qtl_signif.txt", sep="\t")
counts = eqtl['gene_id'].value_counts()
counts.describe()

count    11238.000000
mean      2312.935398
std       1490.079008
min          1.000000
25%       1160.250000
50%       2236.000000
75%       3277.750000
max      10799.000000
Name: gene_id, dtype: float64

In [4]:
eqtl_threshold = counts.quantile(0.75)
eqtl_genes = counts[counts > eqtl_threshold].index

In [5]:
gene_list = high_tpm_var_genes.intersection(set(eqtl_genes))
print(len(gene_list), "candidate genes for enformer prediction experiments")

1358 candidate genes for enformer prediction experiments


In [10]:
with open("/home/s1mi/Github/deep-learning-in-genomics/posts/2023-08-31-Br-personalized-prediction-on-more-genes/gene_list.txt", "r") as f:
    completed_genes = f.read().splitlines()
remaining_genes = [gene for gene in gene_list if gene not in completed_genes]
with open("gene_list_2.txt", "w") as f:
    f.write("\n".join(remaining_genes))

## Run the Pipeline

### Write Metadata


In [11]:
## write intervals.txt
annot_df = pd.read_csv('/home/s1mi/enformer_rat_data/annotation/rn7.gene.txt', sep ='\t',  index_col='geneId')

def write_intervals(gene_list, file):
    with open(file, 'w') as f:
        for gene in gene_list:
            gene_annot = annot_df.loc[gene]
            tss = gene_annot['tss']
            interval = f"chr{gene_annot['chromosome']}_{tss}_{tss}"
            f.write(interval + '\n')

write_intervals(remaining_genes, "metadata/intervals.txt")

/tmp/ipykernel_79532/1904099158.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  annot_df = pd.read_csv('/home/s1mi/enformer_rat_data/annotation/rn7.gene.txt', sep ='\t',  index_col='geneId')



```
mkdir metadata
cp ~/Github/deep-learning-in-genomics/posts/2023-08-31-Br-personalized-prediction-on-more-genes/metadata/individuals.txt metadata
cp ~/Github/deep-learning-in-genomics/posts/2023-08-31-Br-personalized-prediction-on-more-genes/personalized_config.json .
```

(Replace every string `2023-08-31-Br-personalized-prediction-on-more-genes` with `2023-11-02-Br-personalized-prediction-additional-genes`)

### Submit Jobs

```
module load conda
conda activate /lus/grand/projects/TFXcan/imlab/shared/software/conda_envs/enformer-predict-tools

python /home/s1mi/Github/enformer_epigenome_pipeline/enformer_predict.py --parameters /home/s1mi/Github/deep-learning-in-genomics/posts/2023-08-31-Br-personalized-prediction-on-more-genes/personalized_config.json


```

### Submit Jobs

```
module load conda
conda activate /lus/grand/projects/TFXcan/imlab/shared/software/conda_envs/enformer-predict-tools

python /home/s1mi/Github/enformer_epigenome_pipeline/enformer_predict.py --parameters /home/s1mi/Github/deep-learning-in-genomics/posts/2023-08-31-Br-personalized-prediction-on-more-genes/personalized_config.json

```